In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Rondônia - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_RO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - Produção de Cimento (t),Rondônia - IDH Renda,...,Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - Consumo de Cimento (t)
0,2008,0.751494,7.979888,2.208447e+07,1.637885e+06,11.689351,1.870941e+07,0.698932,150.448962,0.693174,...,11.760983,0.655238,22.808561,1006.894941,19176.848685,26.497969,21.685776,75.469241,5.070682,390.788000
1,2009,0.752810,8.008176,2.226473e+07,1.643078e+06,11.684640,1.873529e+07,0.700638,336.177608,0.693109,...,9.472585,0.659243,19.588422,1006.775008,19127.730770,26.528813,21.707671,75.437690,4.902049,452.325000
2,2010,0.753706,7.971248,2.285838e+07,1.640521e+06,11.724602,1.891099e+07,0.702254,502.201514,0.693063,...,9.332485,0.663312,18.596388,1006.795758,14826.071459,26.434620,21.583453,75.296199,4.742629,921.425000
3,2011,0.754192,7.956614,2.338128e+07,1.618454e+06,11.764179,1.903017e+07,0.703786,583.009861,0.693036,...,10.985689,0.667449,18.419365,1007.036992,8438.763610,26.447727,21.511515,74.910155,4.560734,1021.570000
4,2012,0.754233,6.546489,2.383098e+07,1.560372e+06,11.757682,1.910530e+07,0.705401,560.443805,0.692995,...,8.159014,0.672215,19.487867,1007.195877,4928.830653,26.509272,21.598374,75.028438,4.420622,933.501000
5,2013,0.753919,6.917221,2.422272e+07,1.518707e+06,11.741719,1.916060e+07,0.706351,460.001047,0.693078,...,7.913561,0.675149,21.002459,1007.408812,3448.198287,26.513099,21.752006,75.695263,4.276194,826.670000
6,2014,0.753246,7.287260,2.458709e+07,1.494999e+06,11.767208,1.922796e+07,0.707964,357.677944,0.693001,...,10.350442,0.680412,20.055465,1007.673651,2597.909114,26.561558,21.940898,76.327127,4.207147,614.394714
7,2015,0.752267,7.654493,2.486993e+07,1.466317e+06,11.788091,1.927573e+07,0.709294,276.938163,0.693168,...,12.454398,0.684863,18.767358,1007.697489,2136.164001,26.644505,22.123957,76.795244,4.212726,510.273771
8,2016,0.759242,7.643673,2.507534e+07,1.450033e+06,11.811765,1.931637e+07,0.714910,239.494881,0.693113,...,13.111908,0.695058,17.848044,1007.584830,1863.004107,26.594749,22.202418,77.390294,4.183474,410.580514
9,2017,0.758083,7.282097,2.517074e+07,1.454651e+06,11.835195,1.934371e+07,0.714902,226.902776,0.693760,...,9.479812,0.695427,17.563275,1007.547716,1682.028677,26.577385,22.310278,77.982455,4.075241,406.814000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - Produção de Cimento (t),Rondônia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.751494,7.979888,2.208447e+07,1.637885e+06,11.689351,1.870941e+07,0.698932,150.448962,0.693174,5.483335e+08,...,2.868108,11.760983,0.655238,22.808561,1006.894941,19176.848685,26.497969,21.685776,75.469241,5.070682
1,0.752810,8.008176,2.226473e+07,1.643078e+06,11.684640,1.873529e+07,0.700638,336.177608,0.693109,5.302764e+08,...,2.305885,9.472585,0.659243,19.588422,1006.775008,19127.730770,26.528813,21.707671,75.437690,4.902049
2,0.753706,7.971248,2.285838e+07,1.640521e+06,11.724602,1.891099e+07,0.702254,502.201514,0.693063,5.046425e+08,...,1.958246,9.332485,0.663312,18.596388,1006.795758,14826.071459,26.434620,21.583453,75.296199,4.742629
3,0.754192,7.956614,2.338128e+07,1.618454e+06,11.764179,1.903017e+07,0.703786,583.009861,0.693036,4.854622e+08,...,2.079088,10.985689,0.667449,18.419365,1007.036992,8438.763610,26.447727,21.511515,74.910155,4.560734
4,0.754233,6.546489,2.383098e+07,1.560372e+06,11.757682,1.910530e+07,0.705401,560.443805,0.692995,4.867661e+08,...,2.183282,8.159014,0.672215,19.487867,1007.195877,4928.830653,26.509272,21.598374,75.028438,4.420622
5,0.753919,6.917221,2.422272e+07,1.518707e+06,11.741719,1.916060e+07,0.706351,460.001047,0.693078,5.225848e+08,...,2.403445,7.913561,0.675149,21.002459,1007.408812,3448.198287,26.513099,21.752006,75.695263,4.276194
6,0.753246,7.287260,2.458709e+07,1.494999e+06,11.767208,1.922796e+07,0.707964,357.677944,0.693001,5.676413e+08,...,2.748397,10.350442,0.680412,20.055465,1007.673651,2597.909114,26.561558,21.940898,76.327127,4.207147
7,0.752267,7.654493,2.486993e+07,1.466317e+06,11.788091,1.927573e+07,0.709294,276.938163,0.693168,5.179392e+08,...,2.501529,12.454398,0.684863,18.767358,1007.697489,2136.164001,26.644505,22.123957,76.795244,4.212726
8,0.759242,7.643673,2.507534e+07,1.450033e+06,11.811765,1.931637e+07,0.714910,239.494881,0.693113,4.524144e+08,...,2.073464,13.111908,0.695058,17.848044,1007.584830,1863.004107,26.594749,22.202418,77.390294,4.183474
9,0.758083,7.282097,2.517074e+07,1.454651e+06,11.835195,1.934371e+07,0.714902,226.902776,0.693760,4.213689e+08,...,2.177386,9.479812,0.695427,17.563275,1007.547716,1682.028677,26.577385,22.310278,77.982455,4.075241


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      452.325000
1      921.425000
2     1021.570000
3      933.501000
4      826.670000
5      614.394714
6      510.273771
7      410.580514
8      406.814000
9      376.138000
10     399.441000
11     488.194000
12     502.824000
13     501.142000
14            NaN
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - Produção de Cimento (t),Rondônia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.366933,1.128601,-1.793068,1.343669,-1.602309,-1.897543,-1.597405,-1.383438,0.095187,1.204752,...,2.026352,0.984445,-1.605695,2.457245,-1.212494,1.862951,-0.705394,-0.762192,-0.756764,1.972247
1,-0.774200,1.188885,-1.627168,1.412024,-1.699639,-1.776559,-1.284617,-0.064285,-0.087133,0.820657,...,0.085558,-0.051710,-1.324921,0.144226,-1.582145,1.855564,-0.347758,-0.699954,-0.781667,1.489960
2,-0.370197,1.110188,-1.080790,1.378366,-0.873963,-0.955015,-0.988265,1.114914,-0.215819,0.275392,...,-1.114490,-0.115145,-1.039610,-0.568350,-1.518190,1.208644,-1.439934,-1.053053,-0.893344,1.034020
3,-0.151306,1.079003,-0.599524,1.087914,-0.056217,-0.397801,-0.707210,1.688862,-0.291249,-0.132596,...,-0.697344,0.633403,-0.749553,-0.695505,-0.774675,0.248067,-1.287958,-1.257539,-1.198046,0.513803
4,-0.132584,-1.926102,-0.185632,0.323418,-0.190465,-0.046510,-0.410992,1.528584,-0.406659,-0.104860,...,-0.337667,-0.646476,-0.415389,0.071997,-0.284967,-0.279787,-0.574331,-1.010638,-1.104686,0.113084
5,-0.274035,-1.136039,0.174914,-0.224995,-0.520296,0.212086,-0.236740,0.815181,-0.174695,0.657046,...,0.422337,-0.757614,-0.209634,1.159926,0.371328,-0.502457,-0.529965,-0.573928,-0.578367,-0.299977
6,-0.577490,-0.347451,0.510268,-0.537052,0.006355,0.527031,0.059020,0.088423,-0.389665,1.615453,...,1.613108,0.345772,0.159340,0.479702,1.187601,-0.630330,0.031926,-0.036991,-0.079642,-0.497452
7,-1.018893,0.435155,0.770583,-0.914581,0.437846,0.750387,0.303008,-0.485038,0.076848,0.558230,...,0.760923,1.298413,0.471418,-0.445543,1.261073,-0.699772,0.993697,0.483365,0.289839,-0.481495
8,2.124681,0.412096,0.959640,-1.128914,0.927003,0.940414,1.333009,-0.750982,-0.076826,-0.835563,...,-0.716758,1.596124,1.186272,-1.105884,0.913840,-0.740852,0.416771,0.706397,0.759507,-0.565156
9,1.602027,-0.358454,1.047447,-1.068126,1.411104,1.068244,1.331508,-0.840419,1.732167,-1.495939,...,-0.358019,-0.048438,1.212138,-1.310433,0.799449,-0.768068,0.215438,1.012997,1.226895,-0.874702


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.11079346, -0.82973435,  0.81755237, -0.70869537,
          0.86489319,  0.68245254,  0.93691966, -0.75861754,
         -1.02376433, -1.0453709 ,  1.21859064, -0.99976018,
          0.79428371,  1.15575749, -0.52559858,  1.30260559,
         -0.55253046, -0.25130359,  1.38787176,  1.47154451,
         -1.80649616,  1.09296291,  0.94281138,  0.52836084,
          0.61258103,  0.87656257,  1.47801536,  1.71982086,
          1.28221181,  1.39086472,  1.58209844,  1.08872091,
          0.55212962, -1.81398118,  1.44113674,  1.11191094,
          1.30470835,  0.86390516, -1.67365968, -1.70738939,
         -1.55813196, -0.48498301, -0.75714639,  0.40770893,
         -0.27394734,  2.02323219,  1.2908951 ,  1.70465513,
          1.81148084,  1.39968653, -1.36191442,  1.28024593,
          1.11054795,  0.12662799,  1.40414112,  1.40468999,
          1.45643959,  1.58509336,  1.32283157,  1.30222047,
          1.55138602,  1.91801602, -0.77489784,  2.32098884,
         -1.1942541 , -1

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0      452.325000
1      921.425000
2     1021.570000
3      933.501000
4      826.670000
5      614.394714
6      510.273771
7      410.580514
8      406.814000
9      376.138000
10     399.441000
11     488.194000
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    488.194
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - Produção de Cimento (t),Rondônia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-0.970117,-1.191413,0.552069,-0.469187,0.356059,0.388444,0.645784,-0.660402,-2.881361,1.108405,...,-1.482779,-2.260447,0.964061,-2.07127,-0.395184,-0.716176,2.608857,1.699552,1.207817,-1.029715


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[-0.97011657, -1.1914133 ,  0.55206853, -0.46918735,
          0.35605886,  0.38844425,  0.64578366, -0.66040246,
         -2.88136138,  1.10840491,  2.63582309, -0.18995911,
          0.72640489,  2.66833308, -0.032632  ,  0.98802435,
          0.24980494, -0.08955808,  1.85434394,  2.02706568,
         -2.21150703,  1.42938591,  0.85132668,  0.14092386,
          0.30902324,  1.00267904,  2.27970137,  1.11272593,
          0.61007902,  0.89064538,  1.42231438,  1.02650562,
          1.47127593, -1.53201473,  1.73503858,  1.66610613,
          1.0456894 ,  0.55289259, -1.81216593, -2.10342296,
         -2.55377478,  0.41940628, -1.75756445,  0.241023  ,
         -0.98659345,  1.52184916,  1.01398607,  1.49290967,
          1.40764092,  1.87930411, -2.51892072,  2.83356214,
          1.58620924, -0.02524128,  1.54231542,  1.5176119 ,
          1.7355269 ,  1.87941982,  1.54198129,  1.51276712,
          1.83795594,  2.62210693,  1.04993174,  0.96476493,
         -1.48277861, -2

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    502.824
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [18]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [20]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[440867234, 1323760857, 3496926560, 4272221131, 4276823770, 1727651934, 3558824498, 2502597400, 1584648047, 987756842, 4044133582, 3312825705, 1116505363, 716861068, 1269268936, 3209467509, 2065751633, 3442473928, 108361124, 3898026628, 1541875484, 3577535649, 2416206017, 1616625533, 951630229, 820502244, 1332154943, 688095314, 2748230931, 2640614042, 940400343, 225484751, 1832284295, 2847924152, 2475617294, 797187576, 2379764691, 982626690, 2267539335, 3117708278, 4249063028, 4131619829, 995547505, 917373424, 1324167446, 708482456, 725079758, 3761173041, 3211868183, 1481008151, 3402428031, 1348166348, 1763632054, 55694773, 369912360, 1759312496, 1436271827, 2137626871, 2111993365, 2023737702, 1682832058, 2468497874, 634730457, 274539959, 2242138403, 1977951175, 1508745694, 875422843, 1050696780, 3812298902, 580863045, 47504280, 3635322474, 4235112852, 3200847524, 1392214501, 2786590751, 1446551911, 3318551767, 4251946663, 2559300633, 1686073990, 4126927589, 3433889830, 4185813766, 311

2023-09-10 10:34:55.956870: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-10 10:34:55.972134: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-10 10:34:55.972360: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

loss: 9380.0634765625
winner_seed: 440867234


Step: 1 ___________________________________________
loss: 9271.5146484375
winner_seed: 1323760857


Step: 2 ___________________________________________
loss: 8746.1513671875
winner_seed: 3496926560


Step: 3 ___________________________________________
loss: 6578.97021484375
winner_seed: 4272221131


Step: 4 ___________________________________________
loss: 7936.94189453125


Step: 5 ___________________________________________
loss: 8393.2705078125


Step: 6 ___________________________________________
loss: 8040.67333984375


Step: 7 ___________________________________________
loss: 9839.66015625


Step: 8 ___________________________________________
loss: 8108.35400390625


Step: 9 ___________________________________________
loss: 8590.8095703125


Step: 10 ___________________________________________
loss: 7901.751953125


Step: 11 ___________________________________________
loss: 10091.2421875


Step: 12 ___________________________________

In [21]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 430276.5312
Epoch 2/10000
1/1 [==============================] - 0s 5ms/step - loss: 412728.3750
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 297254.9688
Epoch 4/10000
1/1 [==============================] - 0s 4ms/step - loss: 54696.2930
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 139142.5781
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 28418.2949
Epoch 7/10000
1/1 [==============================] - 0s 3ms/step - loss: 34972.5117
Epoch 8/10000
1/1 [==============================] - 0s 3ms/step - loss: 28314.4121
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 23327.7812
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 23074.2656
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 20859.8438
Epoch 12/10000
1/1 [==============================] - 0s 4ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 24925.3223
Epoch 98/10000
1/1 [==============================] - 0s 4ms/step - loss: 19975.3652
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 19177.8477
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 15714.3135
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 20913.3262
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 16404.6387
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 22159.1152
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 18565.7988
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 16312.8594
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 19106.0547
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 19715.8535
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 17208.8184
Epoch 193/10000
1/1 [==============================] - 0s 4ms/step - loss: 13906.3818
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 23457.0020
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 21937.7500
Epoch 196/10000
1/1 [==============================] - 0s 5ms/step - loss: 18893.2871
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 17927.1582
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 12835.9658
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 10613.9609
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 23709.3379
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 22573.6816
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 20384.4062
Epoch 203/10000
1/1 [==============================] - 0s 5ms/step - l

1/1 [==============================] - 0s 6ms/step - loss: 16739.6504
Epoch 288/10000
1/1 [==============================] - 0s 6ms/step - loss: 21640.0645
Epoch 289/10000
1/1 [==============================] - 0s 5ms/step - loss: 18639.3516
Epoch 290/10000
1/1 [==============================] - 0s 6ms/step - loss: 17250.5547
Epoch 291/10000
1/1 [==============================] - 0s 5ms/step - loss: 19982.0879
Epoch 292/10000
1/1 [==============================] - 0s 6ms/step - loss: 23135.5781
Epoch 293/10000
1/1 [==============================] - 0s 5ms/step - loss: 19450.8340
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 15776.1836
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 13752.7031
Epoch 296/10000
1/1 [==============================] - 0s 8ms/step - loss: 12653.4482
Epoch 297/10000
1/1 [==============================] - 0s 5ms/step - loss: 18851.2871
Epoch 298/10000
1/1 [==============================] - 0s 5ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 16102.6143
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 16508.6719
Epoch 384/10000
1/1 [==============================] - 0s 7ms/step - loss: 19065.6504
Epoch 385/10000
1/1 [==============================] - 0s 5ms/step - loss: 19908.4609
Epoch 386/10000
1/1 [==============================] - 0s 7ms/step - loss: 10680.1709
Epoch 387/10000
1/1 [==============================] - 0s 6ms/step - loss: 19891.7715
Epoch 388/10000
1/1 [==============================] - 0s 7ms/step - loss: 17197.3516
Epoch 389/10000
1/1 [==============================] - 0s 7ms/step - loss: 17989.9258
Epoch 390/10000
1/1 [==============================] - 0s 5ms/step - loss: 14716.8320
Epoch 391/10000
1/1 [==============================] - 0s 6ms/step - loss: 15110.3643
Epoch 392/10000
1/1 [==============================] - 0s 5ms/step - loss: 12583.7695
Epoch 393/10000
1/1 [==============================] - 0s 6ms/step - l

...

1/1 [==============================] - 0s 4ms/step - loss: 22690.7578
Epoch 1141/10000
1/1 [==============================] - 0s 4ms/step - loss: 20921.3613
Epoch 1142/10000
1/1 [==============================] - 0s 4ms/step - loss: 11786.7305
Epoch 1143/10000
1/1 [==============================] - 0s 4ms/step - loss: 18211.6973
Epoch 1144/10000
1/1 [==============================] - 0s 4ms/step - loss: 27021.3574
Epoch 1145/10000
1/1 [==============================] - 0s 4ms/step - loss: 12681.1455
Epoch 1146/10000
1/1 [==============================] - 0s 4ms/step - loss: 25091.9043
Epoch 1147/10000
1/1 [==============================] - 0s 4ms/step - loss: 21802.3984
Epoch 1148/10000
1/1 [==============================] - 0s 4ms/step - loss: 21551.2070
Epoch 1149/10000
1/1 [==============================] - 0s 4ms/step - loss: 19075.3223
Epoch 1150/10000
1/1 [==============================] - 0s 4ms/step - loss: 14182.1201
Epoch 1151/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 5ms/step - loss: 22651.5332
Epoch 1235/10000
1/1 [==============================] - 0s 5ms/step - loss: 17440.7891
Epoch 1236/10000
1/1 [==============================] - 0s 4ms/step - loss: 17793.8281
Epoch 1237/10000
1/1 [==============================] - 0s 4ms/step - loss: 17420.3496
Epoch 1238/10000
1/1 [==============================] - 0s 5ms/step - loss: 16134.5010
Epoch 1239/10000
1/1 [==============================] - 0s 5ms/step - loss: 20814.8887
Epoch 1240/10000
1/1 [==============================] - 0s 5ms/step - loss: 23297.9766
Epoch 1241/10000
1/1 [==============================] - 0s 4ms/step - loss: 17212.6895
Epoch 1242/10000
1/1 [==============================] - 0s 5ms/step - loss: 17384.3809
Epoch 1243/10000
1/1 [==============================] - 0s 4ms/step - loss: 21110.0410
Epoch 1244/10000
1/1 [==============================] - 0s 4ms/step - loss: 17558.4238
Epoch 1245/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 4ms/step - loss: 16987.8516
Epoch 1329/10000
1/1 [==============================] - 0s 4ms/step - loss: 19723.1660
Epoch 1330/10000
1/1 [==============================] - 0s 4ms/step - loss: 17930.4941
Epoch 1331/10000
1/1 [==============================] - 0s 4ms/step - loss: 11670.6357
Epoch 1332/10000
1/1 [==============================] - 0s 4ms/step - loss: 12203.6641
Epoch 1333/10000
1/1 [==============================] - 0s 4ms/step - loss: 25138.0020
Epoch 1334/10000
1/1 [==============================] - 0s 5ms/step - loss: 17687.5566
Epoch 1335/10000
1/1 [==============================] - 0s 6ms/step - loss: 18141.4824
Epoch 1336/10000
1/1 [==============================] - 0s 4ms/step - loss: 14690.1201
Epoch 1337/10000
1/1 [==============================] - 0s 4ms/step - loss: 16012.6758
Epoch 1338/10000
1/1 [==============================] - 0s 4ms/step - loss: 13423.3623
Epoch 1339/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 4ms/step - loss: 14849.5205
Epoch 1423/10000
1/1 [==============================] - 0s 4ms/step - loss: 22366.4395
Epoch 1424/10000
1/1 [==============================] - 0s 4ms/step - loss: 16848.4336
Epoch 1425/10000
1/1 [==============================] - 0s 4ms/step - loss: 19820.9512
Epoch 1426/10000
1/1 [==============================] - 0s 4ms/step - loss: 13299.3281
Epoch 1427/10000
1/1 [==============================] - 0s 4ms/step - loss: 21368.9785
Epoch 1428/10000
1/1 [==============================] - 0s 4ms/step - loss: 13471.4844
Epoch 1429/10000
1/1 [==============================] - 0s 4ms/step - loss: 16470.8145
Epoch 1430/10000
1/1 [==============================] - 0s 5ms/step - loss: 14272.2158
Epoch 1431/10000
1/1 [==============================] - 0s 4ms/step - loss: 16289.5420
Epoch 1432/10000
1/1 [==============================] - 0s 4ms/step - loss: 20995.9570
Epoch 1433/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 5ms/step - loss: 20757.4863
Epoch 1517/10000
1/1 [==============================] - 0s 7ms/step - loss: 16256.1797
Epoch 1518/10000
1/1 [==============================] - 0s 5ms/step - loss: 15283.4346
Epoch 1519/10000
1/1 [==============================] - 0s 6ms/step - loss: 20155.4238
Epoch 1520/10000
1/1 [==============================] - 0s 6ms/step - loss: 20250.5371
Epoch 1521/10000
1/1 [==============================] - 0s 4ms/step - loss: 19358.8203
Epoch 1522/10000
1/1 [==============================] - 0s 4ms/step - loss: 15390.3271
Epoch 1523/10000
1/1 [==============================] - 0s 4ms/step - loss: 17950.2324
Epoch 1524/10000
1/1 [==============================] - 0s 4ms/step - loss: 25270.6328
Epoch 1525/10000
1/1 [==============================] - 0s 5ms/step - loss: 22781.4141
Epoch 1526/10000
1/1 [==============================] - 0s 7ms/step - loss: 16250.3057
Epoch 1527/10000
1/1 [==============================] - 0s 7

In [22]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 580ms/step
[502.824](test_target) - [474.77783](prediction) = 28.046167968750012


In [23]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.05577730571482271

In [24]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [25]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - Produção de Cimento (t),Rondônia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.294335,-0.415281,-0.961963,-1.230898,-0.570897,-0.846639,-1.235610,-1.246962,1.288777,1.148205,...,1.308949,1.412346,-1.221460,1.377871,1.395565,0.719114,0.276338,0.495192,0.906319,1.236047
1,0.153687,1.378391,-0.416778,1.218498,-0.835068,-0.557701,0.022028,0.045715,-0.140116,0.140893,...,-0.190804,-0.643250,-0.006544,-0.413067,-0.896020,0.695031,1.062967,0.899613,0.487021,-0.022926
2,1.140648,-0.963110,1.378740,0.012400,1.405965,1.404340,1.213582,1.201247,-1.148661,-1.289098,...,-1.118144,-0.769096,1.228004,-0.964804,-0.499545,-1.414145,-1.339305,-1.394805,-1.393340,-1.213121


train_target:


,Rondônia - Consumo de Cimento (t)
0,452.325
1,921.425
2,1021.570


test_input:


...

,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - Produção de Cimento (t),Rondônia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-0.970117,-1.191413,0.552069,-0.469187,0.356059,0.388444,0.645784,-0.660402,-2.881361,1.108405,...,-1.482779,-2.260447,0.964061,-2.07127,-0.395184,-0.716176,2.608857,1.699552,1.207817,-1.029715


test_target:


,Rondônia - Consumo de Cimento (t)
12,502.824


1/1 [==============================] - 1s 562ms/step
Error: 28.046167968750012




[975.3545532226562,
 940.8942260742188,
 746.1443481445312,
 562.1318359375,
 505.79193115234375,
 412.65545654296875,
 434.9330749511719,
 389.98394775390625,
 394.3988342285156,
 474.77783203125]

In [26]:
display(targets)
display(predictions)

[933.501,
 826.67,
 614.3947142857144,
 510.2737714285714,
 410.5805142857144,
 406.814,
 376.138,
 399.441,
 488.194,
 502.824]

[975.3545532226562,
 940.8942260742188,
 746.1443481445312,
 562.1318359375,
 505.79193115234375,
 412.65545654296875,
 434.9330749511719,
 389.98394775390625,
 394.3988342285156,
 474.77783203125]

In [27]:
mae = mean_absolute_error(predictions, targets)
mae

63.083181201171875

In [28]:
porcentage = mae/np.mean(targets)
porcentage

0.11535039426373182